In [ ]:
%%capture
!pip install -U datasets
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install -q transformers


In [ ]:
import os


In [ ]:
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer, Wav2Vec2Processor

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
!pwd
%cd gdrive/MyDrive/


/content
/content/gdrive/MyDrive


In [ ]:
%cd audio

[Errno 2] No such file or directory: 'audio'
/content/gdrive/MyDrive


In [ ]:
cd audio

/content/gdrive/MyDrive/audio/audio


In [ ]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("Harveenchadha/vakyansh-wav2vec2-tamil-tam-250")
model = Wav2Vec2ForCTC.from_pretrained("Harveenchadha/vakyansh-wav2vec2-tamil-tam-250")

Downloading:   0%|          | 0.00/576 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:423: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,


Downloading:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

In [ ]:
audio_file = "./M_test_final.wav"

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\<\>\s]'

def remove_special_characters(transcription):
    pat = r'[<s>]' 
    return re.sub(pat, '', transcription)

In [ ]:
!pip install SpeechRecognition
!pip install pydub

     |████████████████████████████████| 32.8 MB 40 kB/s 


In [ ]:
import speech_recognition as sr 
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence


r = sr.Recognizer()


def asr_transcript1(tokenizer, model, audio_file):
    sound = AudioSegment.from_wav(audio_file)  
    chunks = split_on_silence(sound,
        min_silence_len = 500,
        silence_thresh = sound.dBFS-14,
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    transcript1 = ""
    for i, audio_chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            speech, rate = librosa.load(chunk_filename,sr=16000)
            try:
                input_values = tokenizer(speech, return_tensors= 'pt').input_values
                logits = model(input_values).logits
                predicted_ids1 = torch.argmax(logits, dim=-1)
                transcription1 = tokenizer.decode(predicted_ids1[0])
                remove_trans=remove_special_characters(str(transcription1))
            except sr.UnknownValueError as e:
                 print("Error:", str(e))
            else:
                print(chunk_filename, ":", remove_trans)
                transcript1 += remove_trans
    return transcript1

In [ ]:
path = asr_transcript1(tokenizer, models, audio_file)
print(path)

audio-chunks/chunk1.wav : ஹலோ ஹலோ 
audio-chunks/chunk2.wav : ஹலோ ஹலோ சார் ஹேப்பி ஆப்னூன் தமிழ் மேட்ரி மூணுல இருந்து தார்னி பேசுறேன் சார் 
audio-chunks/chunk3.wav : சொல்லுங்க மேடம் 
audio-chunks/chunk4.wav : சார் ஆன்லைன்ல ரெஜிஸ்டர் பணி ஹாலைன்ஸ் பாத்துட்டு இருக்கீங்கங்களா சார் தண்டபாணி சார் 
audio-chunks/chunk5.wav : ஆமம்ம்சார் நம்ம ட்ரைட்ல ப்ரொபைல் ரிடிஸ்ர் பண்ணதுக்கு ரொம்ப நன்றி சார்க்கு சீக்கிரம் திருமணமாக வாழ்த்துக்கள் 
audio-chunks/chunk6.wav : இது ஒரு வெல்க்கம் கால் தான் நம்மளோட ப்ரொஃபைல் பத்தி உங்க கிட்ட ஒரு ஃபைவ் மினிட்ஸ் பேசனால சார் சொல்லுங்க மேடம் சொல்லுங்க மேடம் 
audio-chunks/chunk7.wav : சசார தண்டபானி சார்தான் பேசுறாதுங்களா ஆா மேம் மேடம் 
audio-chunks/chunk8.wav : ஏன் ஓகேங்க சார் நம்மளோட ப்ரொஃபைல் செக் பண்ண 
audio-chunks/chunk9.wav : அத பாத்தீங்கன்னா நம் ப்ரொபைல் இன்னு புல்லா கம்ப்ளீட்னெஸ் பண்ணாம இருக்கும் நம்ளோட போட்டோ சாதகம் பார்னர் ப்ரஸபன்ிங் இதெல்லாமே ஆட் பண்ணாம இருக்கீங்க ஏ சார் 
audio-chunks/chunk10.wav : இல்ல மேடம் எனக்கு கல்புலச்நான் பிசினஸ் மன் மேடம் எனக்கு அல்கல டைம

In [ ]:
#groundTruth="வணக்கம் நம்ம இன்னைக்கு நல்ல ஒரு திருக்குறளை பத்தி பாக்க போறோம் இந்த திருக்குறள் என்னென்ன வந்து எப்பொருள் யார்யார்வாய்க் கேட்பினும் அப்பொருள் மெய்ப்பொருள் காண்பது அறிவு இந்த குறளுடைய ஒரு ஒரு வார்த்தையும் வந்து இன்னைக்கு அர்த்தமா பார்ப்போம் எப்பொருள் அதாவது யார்ட்ட இருந்து எந்த விதமான ஒரு விடயத்தை வந்து நம்ம கேட்டு தெரிந்து கொண்டாலும் வந்து அந்த விடயத்தை வந்து நம்ம அப்படியே வந்து உள்வாங்கி எடுத்துக்க கூடாது ஏன்னா வந்து மனித வாழ்க்கை வந்து பலவிதமான அனுபவங்கள் மூலமாக செதுக்கப்பட்டிருக்கு அப்டி இறுக்கப்ப வந்து ஒரு ஒருத்தருடைய அறிவும் வந்து ஒரு வகைப்பட்டது அப்டிறுக்கப்ப ஒருத்தர் சொல்றது வந்து நம்மை ஏன் வந்து அப்படியே வந்து உள்வாங்கி  எடுத்துக்கணும் அப்படி வந்து எடுத்துக்ரா வேண்டிய அவசியம் இல்லை அதுக்காகத்தான் மனிதருக்கு வந்து பகுத்தறிவு அப்படிங்கற ஒரு ஆறாம் அறிவு கொடுக்கப்பட்டிருக்கும் அதையே இந்த திருக்குறள் மிகவும் வலியுறுத்திச்சொல்லுது எப்பொருள் யார் யார் வாய்க் கேட்பினும் அப்ப அதாவது பலவிதமான ஆளுங்ககிட்டருந்து  எந்தவிதமான விஷயத்தை வந்து நம்ம காது வெளியே கேட்டாலும் அதைவந்து நம்ம உள்வாங்கி அதவந்து நம்மளுடைய நோக்கு அல்லது நம்முடைய அனுபவங்கள் மூலமாக வந்து பார்த்துஅதுக்கு ஒரு விடை காண்பது மட்டுமே வந்து நல்ல வழி அப்படிங்கறது தான் வந்து இந்த திருக்குறள் நமக்கு எடுத்துரைக்கிறது வணக்கம்."

In [ ]:
#groundTruth="ஹலோ ஹலோ ஹலோ ஹேப்பிஆப்டர்நூன் தமிழ் மேட்ரிமோனியில இருந்து தாரணி பேசுறேன் சார் சொல்லுங்க மேடம். சார் ஆன்லைன்ல ரெஜிஸ்டர் பண்ணி அலைன்ஸ் பார்த்துட்டு இருக்கீங்க தண்டபாணி சார். ஆமா மேடம். நம்ம வெப்சைட் ரேஜிஸ்டேர் பண்ணதுக்கு  ரொம்ப நன்றி, சார் சீக்கிரமாக திருமணமாக வாழ்த்துக்கள். இது ஒரு வெல்கம் கால் தான். உங்களோட ப்ரொபைல் பத்தி உங்கள்ட ஒரு பைவ் மினிட்ஸ் பேசலாம் இல்ல சார். சொல்லுங்க மேடம், சொல்லுங்க, தண்டபாணி சார்தான் பேசுறதுங்கலா, ஆமா மேடம், ஓகே சார், நம்மளோட ப்ரொபைல் செக் பண்ணேன். அதுல பார்த்தீங்கன்னா நம்மளோட ப்ரொபைல் இன்னும் கம்ப்ளீட் பண்ணாம இருக்கும்.  நம்மளோட போட்டோ ஜாதகம் பார்ட்னர் பிரெபெரென்ஸ் இதெல்லாம் இன்னும் ஆட் பண்ணாம இருக்கீங்க, ஏன் சார், இல்ல மேடம் எனக்கு அதுக்குள்ள நான் பிசினஸ் மென் மேடம் எனக்கு அதுக்குள்ள டயமே இல்லாம போய்டுச்சு, கிளம்பிட்டேன் ஆபீஸ் கிளம்பிட்டேன், அதனாலங்கிள, சரி ஓகே ஓக்கேங்க சார், பிரீ டைம்ல ப்ரொபைல் எல்லாம் ஆட் பண்ணி கம்ப்ளீட் பண்ணிடுங்க, சரி மேடம், அதே மாதிரி மத்த டீடெயில்ஸ்ம் இன்னும் ஆட் பண்ணாம இருக்கீங்க சார், அதே போல நம்ம எக்ஸ்பிஎக்ஷன் எப்படி சார் இருக்கும். நீங்க என்ன மாதிரி வரன் சார் பார்த்துட்டு இருக்கீங்க, எனக்கு ஏஜ் வந்து தர்டி பைவ், அதான் மேடம், அந்த ஏஜ்ல, ஓகே நமக்கு கேஸ்ட் ஒரு ப்ரோப்லேம் கெடயாது, ஓக்கேங்க சார், மாசம் ஒரு பைவ் டு செவன் லாக்ஸ் இன்கமே வருது, ஓக்கேங்க சார், அதான் மேடம், அதாவது ஆப்போசிட் சைடு என்ன படிச்சிருந்தாலும் எனி டிக்ரீக்கு பார்த்துட்டு இருக்கீங்க பிளஸ் கம்யூனிட்டி பார்த்துட்டு இருக்கீங்க, மேடம் படிச்சதுலம் வேணாம் நமக்கு அந்த மாதிரிலாம் வேணாம், நல்ல பேமிலி கேர்ள இருக்கனும் அவ்ளோதான், ஒகே சார், லொகேஷன் என்ன மாதிரி லொகேஷன் சார் பார்த்துட்டு இருக்கீங்க, சென்னை அது இல்லனா மதுரை, சென்னை அது இல்லனா மதுரை, அதே மாதிரி ஏஜ் வந்து தர்டி எயிட் குள்ள பாக்கிறிங்களா, தர்டி தர்டி ஒன், ஓகேங்க சார், இந்த டீடெயில்ஸ் எல்லாமே நமக்கு எடிட் பிரெபெரென்ஸ் ஆப்சன் இருக்கும், நீங்க அதுல அட் பண்ணிக்கலாம், அதே மாதிரி நம்ம டீடெயில்ஸ் செக் பண்ண, நேம் வந்து தண்டபாணி குடுத்திருக்கிங்க, ஏஜ் வந்து தர்டி பைவ் குடுத்திருக்கிங்க, லொகேஷன் திண்டுக்கல் குடுத்திருக்கிங்க, பிஸ்னஸ் பண்றத குடுத்திருக்கிங்க, எஸ் சி கம்யூனிட்டி, செவ்வாய் தோஷம் இருக்குனு குடுத்திருக்கிங்க, அய்யயோ அது இல்ல மேடம் நான் அன்னைக்கு தப்ப கொடுத்திருக்கேன், எனக்கு எந்த தோஷம் கெடயாது மேடம், ஒகே சார் ஒன்னும் ப்ரோப்லேம் இல்ல, எடிட் பண்ணிக்கலாம் சார், நீங்க எடிட் பண்ணிக்குங்க, அதே மாதிரி பிசினஸ் என்ன சார் பண்ணிட்டு இருக்கீங்க, பைனான்ஸ், ஓகே சார், பேமிலி டீடெயில்ஸ் ஆட் பண்ணாம இருக்கீங்க சார், பாதரோட ஆக்குபேஷன், மதர் என்ன பன்றாங்க, சிஸ்டேர்ஸ் அண்ட் பிரோதெரஸ், அப்பா கிடையாது, பிரதர் ரெண்டு பிரதர் இருகாங்க, சரிங்க சார், சார் அதே மாதிரி ஆனுவல் இன்கம் பார்த்தீங்கன்னா நைண்ட்டி டு ஒன்."

In [ ]:
groundTruth_Modified="வணக்கம் நம்ம இன்னைக்கு ஒரு நல்ல ஒரு திருக்குறல பத்தி பார்க்க போறோம் இந்த திருக்குறள் என்னன்னா வந்து ஆ் எப்பொருள் யார் யார் வாய்க் கேட்பினும் அப்பொருள் மெய்ப்பொருள் காண்பது அறிவு இந்த குறளுடைய ஒரு ஒரு வார்த்தையும் வந்து நம்ன்னிக்கை அர்த்தமா பாப்போம் எப்பொருள் அதாவது யார்ட்டருந்து எந்த விதமான ஒரு விடயத்த வந்து நம்ம கேட்டு தெரிஞ்சுகிட்டாலும் வந்து அந்த விடயத்த வந்து நம்ம அப்படியே வந்து உள்வாங்கி எடுத்துக்கக் கூடாது ஏன்ன வந்து மனித வாழ்க்கை வந்து பல விதமான அனுபவங்கள் மூலமாதான் செதுக்கப்பட்டிருக்கு அப்படி இருக்கிறப்பு வந்து ஒரு ஒருத்துருடைய அறிவுவந்து ஒருரு வகைப்பட்டது அப்படிதறுப்பு ஒருத்தர் சொல்றது வந்து நம்ம ஏன் வந்து அப்படியே வந்து உள்வாங்கி எடுத்துக்கணும் அப்படி வந்து எடுத்துக்கிறவாடிய அவசியம் இல்ல அதுக்கா்தான் மனிதருக்கு வந்து பகுத்தறிவு அப்படிங்ற ஒரு ஆறாம் அறிவு கொடுக்கப்பட்டருக்கு அதையே இந்த திருக்குறல் மிகவும் வலியுறுத்திச் சொல்லுது எப்பொருள் யார்யார்வாய் கேட்பினும் அதாவது பலவிதமான ஆளுங்கட்டந்து எந்த விதமான விஷயத்தை வந்து நம்ம காது வழியே கேட்டாலும் அத வந்து நம உள்வாங்கி அத வந்து நம்மளுடைய நோக்கு அல்லது நம்முடைய அனுபவங்கள் மூலமா வந்து பார்த்து அதுக்கு ஒரு விடை காண்வது மட்டுமே வந்து நல்ல வழி அப்டிங்கிறத தான் வந்து இந்த திருக்குறள் நமக்கு எடுத்துரைக்குது வணக்கம்"

In [ ]:
#groundTruth_Modified ="ஹலோ ஹலோ ஹலோ ஹேப்பிஆப்டர்நூன் தமிழ் மேட்ரிமூணுல இருந்து தார்னி பேசுறேன் சார் சொல்லுங்க மேடம் சார் ஆன்லைன்ல ரெஜிஸ்டர் பண்ணி ஹாலைன்ஸ் பார்த்துட்டு இருக்கீங்க தண்டபாணி சார் ஆமா மேடம். நம்ம வெப்சைட்ல ரேஜிஸ்டேர் பண்ணதுக்கு  ரொம்ப நன்றி சார்க்கு சீக்கிரமாக திருமணமாக வாழ்த்துக்கள் இது ஒரு வெல்கம் கால் தான் உங்களோட ப்ரொபைல் பத்தி உங்கள்ட ஒரு பைவ் மினிட்ஸ் பேசலாம் இல்ல சார் சொல்லுங்க மேடம் சொல்லுங்க தண்டபாணி சார்தான் பேசுறதுங்கலா ஆமா மேடம் ஓகே சார் நம்மளோட ப்ரொபைல் செக் பண்ணேன் அதுல பார்த்தீங்கன்னா நம்ளோட ப்ரொபைல் இன்னும் கம்ப்ளீட் பண்ணாம இருக்கும்  நம்மளோட போட்டோ ஜாதகம் பார்ட்னர் பிரெபெரென்ஸ் இதெல்லாம் இன்னும் ஆட் பண்ணாம இருக்கீங்க ஏ சார் இல்ல மேடம் எனக்கு அதுக்குள்ள நான் பிசினஸ் மென் மேடம் எனக்கு அதுக்குள்ள டயமே இல்லாம போய்டுச்சு கிளம்பிட்டேன் ஆபீஸ் கிளம்பிட்டேன் அதனாலங்கிள சரி ஓகே ஓக்கேங்க சார் பிரீ டைம்ல ப்ரொபைல் எல்லாம் ஆட் பண்ணி கம்ப்ளீட் பண்ணிடுங்க சரி மேடம் அதே மாதிரி மத்த டீடெயில்ஸ்ம் இன்னும் ஆட் பண்ணாம இருக்கீங்க சார் அதே போல நம்ம எக்ஸ்படேஷன்ஸ் எப்படி சார் இருக்கு நீங்க என்ன மாதிரி வரன் சார் பார்த்துட்டு இருக்கீங்க எனக்கு ஏஜ் வந்து தர்டி பைவ் அதான் மேடம் அந்த ஏஜ்ல ஓகே நமக்கு கேஸ்ட் ஒரு பிராபிளம் கெடயாது ஓக்கேங்க சார் மாசம் ஒரு ஃபைவ் டூ செவன் லேக்ஸ் இன்கமே வருது ஓக்கேங்க சார் அதான் மேடம் அதாவது ஆப்போசிட் சைடு என்ன படிச்சிருந்தாலும் எனி டிகிரி பாத்துட்டு ருக்ங்க பிளஸ் கம்யூனிட்டி பார்த்துட்டு இருக்கீங்க மேடம் படிச்சதுலம் வேணாம் நமக்கு அந்த மாதிரிலாம் வேணாம் நல்ல பேமிலி கேர்ள இருக்கனும் அவ்ளோதான் ஒகே சார் உம்ம் லோகேஷன் என்ன மாதிரி லோகேஷன்ல சார் பார்த்துட்டு இருக்கீங்க சென்னை அது இல்லனா மதுரை சென்னை அது இல்லனா மதுரை அதே மாதிரி ஏஜ் வந்து தர்டி ஏஜ்குள பாக்கிறிங்களா தட்டி தட்டி ஒன் ஓகேங்க சார் இந்த டீடெயில்ஸ் எல்லாமே நமக்கு எடிட் பிரெபெரென்ஸ் ஆப்சன் இருக்கும் நீங்க அதுல அட் பண்ணிக்கலாம் அதே மாதிரி நம்ம டீடெயில்ஸ் செக் பண்ண நேம் வந்து தண்டபாணி குடுத்திருக்கிங்க ஏஜ் வந்து தர்டி பைவ் குடுத்திருக்கிங்க லொகேஷன் திண்டுக்கல் குடுத்திருக்கிங்க பிஸ்னஸ் பண்றத குடுத்திருக்கிங்க எஸ் சி கம்யூனிட்டி செவ்வாய் தோஷம் இருக்குனு குடுத்திருக்கிங்க அய்யயோ அது இல்ல மேடம் நான் அன்னைக்கு தப்ப கொடுத்திருக்கேன் எனக்கு எந்த தோஷம் கெடயாது மேடம் ஒகே சார் ஒன்னும் ப்ரோப்லேம் இல்ல எடிட் பண்ணிக்கலாம் சார் நீங்க எடிட் பண்ணிக்கோங்க அதே மாதிரி பிசினஸ் என்ன சார் பண்ணிட்டு இருக்கீங்க பைனான்ஸ் ஓகே சார் பேமிலி டீடெயில்ஸ் ஆட் பண்ணாம இருக்கீங்க சார் ஃபாதரோட ஆக்குபேஷன் மதர் என்ன பன்றாங்க சிஸ்டேர்ஸ் அண்ட் பிரோதெரஸ் அப்பா கிடையாது பிரதர் ரெண்டு பிரதர் இருகாங்க சரிங்க சார் சார் அதே மாதிரி ஆன்வல் இன்கம் பார்த்தீங்கன்னா நைண்ட்டி டூ ஒன்"

In [ ]:
from jiwer import wer
Error_rate = wer(groundTruth_Modified, path)
print(f"WER: {Error_rate}")

WER: 0.6676923076923077


In [ ]:
Error_rate

0.6676923076923077

In [ ]:
file = open('target.txt', 'w')
file.write(path)
file.close()

In [ ]:
file = open('truth.txt', 'w')
file.write(groundTruth)
file.close()

In [ ]:
wer_score = wer(path, groundTruth, standardize=True)

In [ ]:
print(
"Speaker 0 : ஹலோ"'\n'
"Speaker 1 : ஹலோ" '\n'
"Speaker 0 : ஹலோ சார் ஹேப்பி ஆப்னூன் தமிழ் மேட்ரி மூணுல இருந்து தார்னி பேசுறேன் சார்" '\n'
"Speaker 1 : சொல்லுங்க மேடம்" '\n'
"Speaker 0 : சார் ஆன்லைன்ல ரெஜிஸ்டர் பணி ஹாலைன்ஸ் பாத்துட்டு இருக்கீங்கங்களா சார் தண்டபாணி சார்" '\n'
"Speaker 1 : ஆமம்ம்"'\n'
"Speaker 0 : சார் நம்ம ட்ரைட்ல ப்ரொபைல் ரிடிஸ்ர் பண்ணதுக்கு ரொம்ப நன்றி சார்க்கு சீக்கிரம் திருமணமாக வாழ்த்துக்கள் இது ஒரு வெல்க்கம் கால் தான் நம்மளோட ப்ரொஃபைல் பத்தி உங்க கிட்ட ஒரு ஃபைவ் மினிட்ஸ் பேசனால சார்" '\n'
"Speaker 1 : சொல்லுங்க மேடம் சொல்லுங்க மேடம் "'\n'
"Speaker 0 : சசார தண்டபானி சார்தான் பேசுறாதுங்களா" '\n'
"Speaker 1 : ஆா மேம் மேடம் "'\n'
"Speaker 0 : ஓகேங்க சார் நம்மளோட ப்ரொஃபைல் செக் பண்ண அத பாத்தீங்கன்னா நம் ப்ரொபைல் இன்னு புல்லா கம்ப்ளீட்னெஸ் பண்ணாம இருக்கும் நம்ளோட போட்டோ சாதகம் பார்னர் ப்ரஸபன்ிங் இதெல்லாமே ஆட் பண்ணாம இருக்கீங்க ஏ சார் "'\n'
"Speaker 1 : இல்ல மேடம் எனக்கு கல்புலச்நான் பிசினஸ் மன் மேடம் எனக்கு அல்கல டைமிங்லாம்போ தெரிு கெளம்பி ஆஃபீஸ் கெளம்பிட்டேன் "'\n'
"Speaker 0 : அதனாலங்களா சரி ஓகேங்க சார் ஓகேங்க சார் இல்ல ஃப்ரீ டைம்ல இருக்கும்போது ப்ரொஃபைல் எல்லாமே ஆட் பண்ணி கம்ப்ளீட்னெஸ் பண்ணிடுங்க "'\n'
"Speaker 1 : சாரிங் மேடம் "'\n'
"Speaker 0 : அதே மாதிரி மத்த டீடெயில்ஸ இன்னும் ஆட் பண்ணாம இருக்கீங்க சார் நம்மளோட எக்ஸ்படேஷன்ஸ் எல்லாம எப்படி சார் இருக்கும் நீங்க என்ன மாதிரி வரேன் சார் பாத்துட்டு இருக்கீங்க "'\n'
"Speaker 1 : எனக்கு அது சென்ட ம்எனக்கு என்னந்த வந்து தேர்ட்டி பைவ் "'\n'
"Speaker 0 : ஓகேங்க சார் "'\n'
"Speaker 1 : அதான் மேடம் அந்த இகதுல எனிகு இதான் நான கேஸ் நம் பிராபிளம் கெடையாது"  '\n'
)

Speaker 0 : ஹலோ
Speaker 1 : ஹலோ
Speaker 0 : ஹலோ சார் ஹேப்பி ஆப்னூன் தமிழ் மேட்ரி மூணுல இருந்து தார்னி பேசுறேன் சார்
Speaker 1 : சொல்லுங்க மேடம்
Speaker 0 : சார் ஆன்லைன்ல ரெஜிஸ்டர் பணி ஹாலைன்ஸ் பாத்துட்டு இருக்கீங்கங்களா சார் தண்டபாணி சார்
Speaker 1 : ஆமம்ம்
Speaker 0 : சார் நம்ம ட்ரைட்ல ப்ரொபைல் ரிடிஸ்ர் பண்ணதுக்கு ரொம்ப நன்றி சார்க்கு சீக்கிரம் திருமணமாக வாழ்த்துக்கள் இது ஒரு வெல்க்கம் கால் தான் நம்மளோட ப்ரொஃபைல் பத்தி உங்க கிட்ட ஒரு ஃபைவ் மினிட்ஸ் பேசனால சார்
Speaker 1 : சொல்லுங்க மேடம் சொல்லுங்க மேடம் 
Speaker 0 : சசார தண்டபானி சார்தான் பேசுறாதுங்களா
Speaker 1 : ஆா மேம் மேடம் 
Speaker 0 : ஓகேங்க சார் நம்மளோட ப்ரொஃபைல் செக் பண்ண அத பாத்தீங்கன்னா நம் ப்ரொபைல் இன்னு புல்லா கம்ப்ளீட்னெஸ் பண்ணாம இருக்கும் நம்ளோட போட்டோ சாதகம் பார்னர் ப்ரஸபன்ிங் இதெல்லாமே ஆட் பண்ணாம இருக்கீங்க ஏ சார் 
Speaker 1 : இல்ல மேடம் எனக்கு கல்புலச்நான் பிசினஸ் மன் மேடம் எனக்கு அல்கல டைமிங்லாம்போ தெரிு கெளம்பி ஆஃபீஸ் கெளம்பிட்டேன் 
Speaker 0 : அதனாலங்களா சரி ஓகேங்க சார் ஓகேங்க சார் இல்ல ஃப்ரீ டைம்ல இருக்கும்போது ப்